In [17]:
import json
import requests
import csv
import jsonlines
from flask import Flask, redirect, render_template, request, url_for
import sys
import numpy as np
from collections import defaultdict
import random
import math
import openai
import re
import pandas as pd
import os
from numpy.linalg import norm
import itertools
import string
import torch
from transformers import BertTokenizer, BertModel
import logging

#csv.field_size_limit(sys.maxsize)

In [18]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [19]:
openai.api_key = os.environ['openai_api_key'] # apna api key dalde

In [20]:
data = csv.DictReader(open("../IEEE_papers/Raw_data/IEEE VIS papers 1990-2022 - Main dataset.csv"))

In [21]:
dataset = [datum for datum in data]
# dataset = dataset[0:20]

In [22]:
save_json(dataset, r'../IEEE_papers/processed_data/processed_data.json')

Checking frequency of keywords in Author keywords section of research papers

In [23]:
dat = json.load(open(r'../IEEE_papers/processed_data/processed_data.json'))
freq = {}
for datum in dat:
    keywords_raw = datum['AuthorKeywords'].strip()
    keywords = keywords_raw.split(',')
    for keys in keywords:
        keys.strip()
        if (keys in freq):
            freq[keys] += 1
        else:
            freq[keys] = 1

In [24]:
def call_gpt(messages, model="gpt-3.5-turbo-0613"):
    completions = openai.ChatCompletion.create(
        model=model,
        n=1,
        stop=None,
        temperature = 0.0, # change accordingly
        messages=messages)
    gpt_response = completions['choices'][0]['message']['content'].strip() 
    return gpt_response   

Prompt for extracting triggers from abstract

In [28]:
def extract_events(sentence):
    messages = [
        { 
            "role": "system", 
            "content": """
                A research paper asbtract will be given as user input.
                What is the main research idea of the abstract.
                The research idea should majorly describe the research paper.
                The research idea should be human-readable and less than 5 words. 
                Reply in the format:
                [research idea]
            """
        },
        { "role": "system", "name": "example_user", "content": "The success of DL can be attributed to hours of parameter and architecture tuning by human experts. Neural Architecture Search (NAS) techniques aim to solve this problem by automating the search procedure for DNN architectures making it possible for non-experts to work with DNNs. Specifically, One-shot NAS techniques have recently gained popularity as they are known to reduce the search time for NAS techniques. One-Shot NAS works by training a large template network through parameter sharing which includes all the candidate NNs. This is followed by applying a procedure to rank its components through evaluating the possible candidate architectures chosen randomly. However, as these search models become increasingly powerful and diverse, they become harder to understand. Consequently, even though the search results work well, it is hard to identify search biases and control the search progression, hence a need for explainability and human-in-the-loop (HIL) One-Shot NAS. To alleviate these problems, we present NAS-Navigator, a visual analytics (VA) system aiming to solve three problems with One-Shot NAS; explainability, HIL design, and performance improvements compared to existing state-of-the-art (SOTA) techniques. NAS-Navigator gives full control of NAS back in the hands of the users while still keeping the perks of automated search, thus assisting non-expert users. Analysts can use their domain knowledge aided by cues from the interface to guide the search. Evaluation results confirm the performance of our improved One-Shot NAS algorithm is comparable to other SOTA techniques. While adding Visual Analytics (VA) using NAS-Navigator shows further improvements in search time and performance. We designed our interface in collaboration with several deep learning researchers and evaluated NAS-Navigator through a control experiment and expert interviews."},
        { "role": "system", "name": "example_system", "content": "[automating the search procedure for DNN architectures using Neural Architecture Search (NAS) techniques];"},         
        { "role": "user", "content": f"This is the research paper abstract:{sentence}"},
    ]
    events = call_gpt(messages)
    return events

# [
#     "question_1" : question,
#     "answers": {
#         "answer_1":1,
#         ...,
#     },
#     {
#         "question_1" : question,
#     "answers": {
#         "answer_1":1,
#         ...,
#     },...
# ]

In [26]:
def map_entities_abstract(word1, word2,abs):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art classification model. 
                You will be given a pair of 2 keywords and the research paper abstract containing the keywords as user input.
                Your task is to assign a broad and generic research topic that can represent both the words.
                The topic should not be too specific and should be less than 3 words.
                The topic should be very similar to the concept related to the given keywords.
                Strictly assign one topic to a pair of keywords.
                Remember what type was assigned to each keyword for future references.
                Multiple keywords can belong to one topic. 
                Reply in the format:
                research topic.
            """
        },
        # { "role": "system", "name": "example_user", "content": "[Theory , Theoretical and Empirical Research]"},
        # { "role": "system", "name": "example_system", "content": "[Theoretical Research - Theory , Theoretical and Empirical Research];"},    
        { "role": "user", "content": f"This is the first keyword:{word1}"},   
        { "role": "user", "content": f"This is the second keyword:{word2}"},
        { "role": "user", "content": f"This is the abstract:{abs}"}    
    ]
    events = call_gpt(messages)
    return events

In [27]:
def remove_keywords(keywords):
    messages = [
        { 
            "role": "system", 
            "content": """
                User will provide a list of keywords from research paper abstracts majorly talking about the same concept.
                Remove the keywords from the list that do not belong to the same research area or concept.
                Reply in the format:
                "key word 1, key word 2, ...";
            """
        },
        { "role": "system", "name": "example_user", "content": "[Data Storytelling,Deep Learning,ensemble learning,Tracking   Transformation,Motivated Perception]"},
        { "role": "system", "name": "example_system", "content": "Deep Learning,ensemble learning"},    
        { "role": "user", "content": f"This is the list of keywords:{keywords}"},   
        # { "role": "user", "content": f"This is the abstract:{abs}"}    
    ]
    events = call_gpt(messages)
    return events

Prompt used for openAI embeddings

In [29]:
def map_listEntities(keywords):
    messages = [
        { 
            "role": "system", 
            "content": """ 
                User will provide a list of keywords from research paper abstracts.
                The input will be in the format:
                key word 1, key word 2,...
                Which phrase would best describe the list of keywords.
                The phrase should be very specific and similar to the keywords and less than 5 words.
                If the words are too similar to each other, simply assign one of the words as the topic.
                Strictly assign one topic to a list of keywords.
                Reply in the format:
                research topic;
            """
        },
        { "role": "system", "name": "example_user", "content": "[dashboards,dashboard]"},
        { "role": "system", "name": "example_system", "content": "dashboards"},    
        { "role": "user", "content": f"This is the list of keyword:{keywords}"},      
    ]
    events = call_gpt(messages)
    return events

In [30]:
def map_pairEntities(word1, word2):
    messages = [
        { 
            "role": "system", 
            "content": """
                You are a state of the art classification model. 
                You will be given 2 keywords from a research paper abstract as user input.
                Your task is to assign a generic research topic that can represent both the words.
                The topic should not be too specific and should be less than 3 words.
                The topic should be majorly similar to the 2 given keywords.
                Strictly assign one topic to a pair of keywords.
                Remember what type was assigned to each keyword for future references.
                Multiple keywords can belong to one topic. 
                Reply in the format:
                research topic
            """
        },
        # { "role": "system", "name": "example_user", "content": "[Theory , Theoretical and Empirical Research]"},
        # { "role": "system", "name": "example_system", "content": "[Theoretical Research - Theory , Theoretical and Empirical Research];"},    
        { "role": "user", "content": f"This is the first keyword:{word1}"},   
        { "role": "user", "content": f"This is the second keyword:{word2}"},
        # { "role": "user", "content": f"This is the abstract:{abs}"}    
    ]
    events = call_gpt(messages)
    return events

In [31]:
def merge_sentences(datum_sentences):
    sentence_list = [" ".join(sentence_word_list) for sentence_word_list in datum_sentences] # merge the words into sentences
    paragraph = " ".join(sentence_list)
    return paragraph

In [32]:
def strip_sentence(sentence):
    if sentence.startswith('The article discussed how'):
        stripped_sentence = sentence.replace('The article discussed how', '').strip()
        stripped_sentence = re.sub(",","",stripped_sentence)
    elif sentence.startswith('The article discussed'):
        stripped_sentence = sentence.replace('The article discussed', '').strip()
        stripped_sentence = re.sub(",","",stripped_sentence)
    else:
        print("!!!")
    return stripped_sentence

Extract Triggers using gpt api

In [33]:
dat = json.load(open(r'../IEEE_papers/Result/data.json'))
res_events = []
error_datum = []
for index, datum in enumerate(keys):
    print('{}/{}'.format(index, len(keys)))
    sentence = datum['Abstract'].strip()
    keywords = datum['AuthorKeywords'].strip()
    # sentence = re.sub("-"," ",sentence)
    cl = map_entities_abstract(keys)
    # datum['events'] = cl
res_events.append(cl)
save_json(res_events, r'../IEEE_papers/Events/mapped.json')

NameError: name 'cl' is not defined

Helper Functions

In [34]:
def replace_punctuation_with_whitespace(input_string):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    return input_string.translate(translator)

In [35]:
def get_top_n_values(dictionary, n):
    sorted_items = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    top_n_items = sorted_items[:n]
    return dict(top_n_items)

In [36]:
def calculate_cosine(a,b):
    cosine = np.dot(a,b)/(norm(a)*norm(b))
    return cosine

In [37]:
data = json.load(open(r'../IEEE_papers/Result/data.json'))
res_events = []
error_datum = []
for index, datum in enumerate(data):
    print('{}/{}'.format(index, len(dat)))
    sentence = datum['Abstract'].strip()
    sentence = replace_punctuation_with_whitespace(sentence)
    # sentence = re.sub("-"," ",sentence)
    cl = extract_events(sentence)
    print(cl)
    datum['events'] = cl
    res_events.append(datum)
save_json(res_events, r'../IEEE_papers/Events/events.json')

0/3620
[assessing photosensitive risk in interactive visualizations]
1/3620
[visual analytics tool for exploring data heterogeneity in horizontal federated learning (HFL)]
2/3620
[interactive system for rapid transformation of tabular data]
3/3620
[developing an aesthetic pleasure rating scale for visual data representations]
4/3620
[Automating DNN architecture search]
5/3620
[addressing criticisms and improving the scientific values and rigor of visual analytics (VA) systems]
6/3620
[Comparing progressive bar chart designs for early and accurate decision-making]
7/3620
[presenting and exploring hierarchical tables]
8/3620
[visualization techniques for music mood classification]
9/3620
[data physicalization using kirigami]
10/3620
[visualization platform for task parallel traces]
11/3620
[Design choices influence biases]
12/3620
[Creating a feature-based linearization for visualizing time-dependent scalar fields]
13/3620
[exploring complex Gaussian mixture models]
14/3620
[Beliefs bias

Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600)

Generating openAI embeddings for keywords

In [19]:
no_keywords = json.load(open(r'../IEEE_papers/Result/data.json'))
main_chars = []
for dat in no_keywords:
    main_chars.append(dat['main_participants'])
main_chars = [item.strip() for sublist in main_chars for item in sublist]
main_chars = [replace_punctuation_with_whitespace(main_chars[i]) for i in range(len(main_chars))]
main_chars = [i.lower().strip() for i in main_chars]
main_chars = [re.sub(' +',' ',i) for i in main_chars]
save_json(main_chars, r'../IEEE_papers/Result/list_chars.json')

In [20]:
chars = json.load(open(r'../IEEE_papers/Result/main_participants.json'))
# chars = chars[0:100]
flat_list = [item.strip() for sublist in chars for item in sublist]
flat_list = [replace_punctuation_with_whitespace(flat_list[i]) for i in range(len(flat_list))]
flat_list = [i.lower().strip() for i in flat_list]
flat_list = [re.sub(' +',' ',i) for i in flat_list]
while("" in flat_list):
    flat_list.remove("")
save_json(flat_list, r'../IEEE_papers/Result/flat_list.json')


In [21]:
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [26]:
embedds = json.load(open(r'../IEEE_papers/Result/openAI_embeddings.json'))
for index,i in enumerate(main_chars):
    print('{}/{}'.format(index,len(main_chars)))
    if(i not in embedds.keys()):
        embedds[i] = get_embedding(i)
save_json(embedds, r'../IEEE_papers/Result/OpenAI_emb.json')

0/10542
1/10542
2/10542
3/10542
4/10542
5/10542
6/10542
7/10542
8/10542
9/10542
10/10542
11/10542
12/10542
13/10542
14/10542
15/10542
16/10542
17/10542
18/10542
19/10542
20/10542
21/10542
22/10542
23/10542
24/10542
25/10542
26/10542
27/10542
28/10542
29/10542
30/10542
31/10542
32/10542
33/10542
34/10542
35/10542
36/10542
37/10542
38/10542
39/10542
40/10542
41/10542
42/10542
43/10542
44/10542
45/10542
46/10542
47/10542
48/10542
49/10542
50/10542
51/10542
52/10542
53/10542
54/10542
55/10542
56/10542
57/10542
58/10542
59/10542
60/10542
61/10542
62/10542
63/10542
64/10542
65/10542
66/10542
67/10542
68/10542
69/10542
70/10542
71/10542
72/10542
73/10542
74/10542
75/10542
76/10542
77/10542
78/10542
79/10542
80/10542
81/10542
82/10542
83/10542
84/10542
85/10542
86/10542
87/10542
88/10542
89/10542
90/10542
91/10542
92/10542
93/10542
94/10542
95/10542
96/10542
97/10542
98/10542
99/10542
100/10542
101/10542
102/10542
103/10542
104/10542
105/10542
106/10542
107/10542
108/10542
109/10542
110/10542


In [30]:
openAI_embedds = {}
for index,i in enumerate(main_chars):
    print('{}/{}'.format(index,len(main_chars)))
    embed = get_embedding(i)
    openAI_embedds[i] = embed

save_json(openAI_embedds, r'../IEEE_papers/Result/openAI_embeddings_2.json')

0/10542
1/10542
2/10542
3/10542
4/10542
5/10542
6/10542
7/10542
8/10542
9/10542
10/10542
11/10542
12/10542
13/10542
14/10542
15/10542
16/10542
17/10542
18/10542
19/10542
20/10542
21/10542
22/10542
23/10542
24/10542
25/10542
26/10542
27/10542
28/10542
29/10542
30/10542
31/10542
32/10542
33/10542
34/10542
35/10542
36/10542
37/10542
38/10542
39/10542
40/10542
41/10542
42/10542
43/10542
44/10542
45/10542
46/10542
47/10542
48/10542
49/10542
50/10542
51/10542
52/10542
53/10542
54/10542
55/10542
56/10542
57/10542
58/10542
59/10542
60/10542
61/10542
62/10542
63/10542
64/10542
65/10542
66/10542
67/10542
68/10542
69/10542
70/10542
71/10542
72/10542
73/10542
74/10542
75/10542
76/10542
77/10542
78/10542
79/10542
80/10542
81/10542
82/10542
83/10542
84/10542
85/10542
86/10542
87/10542
88/10542
89/10542
90/10542
91/10542
92/10542
93/10542
94/10542
95/10542
96/10542
97/10542
98/10542
99/10542
100/10542
101/10542
102/10542
103/10542
104/10542
105/10542
106/10542
107/10542
108/10542
109/10542
110/10542


Generate similar words mapping fop openAI embeddings

In [28]:
openAI_embedds = json.load(open(r'../IEEE_papers/Result/OpenAI_emb.json'))
cosine = json.load(open(r'../IEEE_papers/Result/cosine.json'))
list_chars = json.load(open(r'../IEEE_papers/Result/list_chars.json'))
similarity_dict = {}
check=[]
for i in range(len(list_chars)):
    print('{}/{}'.format(i, len(list_chars)))
    similar = []
    res= []
    if (list_chars[i] not in check):
        similar.append(list_chars[i])
        check.append(list_chars[i])
        avg_score = 0.0
    for j in range(i+1,len(list_chars)):
        score = calculate_cosine(openAI_embedds[list_chars[i]],openAI_embedds[list_chars[j]])
        if (score>0.941 and list_chars[j] not in check):
            similar.append(list_chars[j])
            check.append(list_chars[j])
            # print([similar,"^^^^^^^^^^^"])
            res = [calculate_cosine(openAI_embedds[a], openAI_embedds[b]) for idx, a in enumerate(similar) for b in similar[idx + 1:]]
            # print([res,"@@@"])
            avg_score = np.mean(res,dtype=np.float64)
            if(avg_score<0.941):
                # print([flat_list[j],"()()()()()"])
                similar.remove(list_chars[j])
            # print(similar)

    similar = list(set(similar))
    similar = ",".join(similar)
    check = list(set(check))
    new_list = list(similar.split(","))
    if(len(new_list)>1):
            mapping = map_listEntities(new_list)
            similarity_dict[similar] = mapping.lower()
save_json(similarity_dict, r'../IEEE_papers/Result/similarity_dict_final.json')

0/10542


ValueError: shapes (1024,) and (1536,) not aligned: 1024 (dim 0) != 1536 (dim 0)

BERT Embeddings

In [50]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
mp = json.load(open(r'../IEEE_papers/Result/main_participants.json'))
mp=mp[0:100]
model = BertModel.from_pretrained('bert-base-uncased',output_hidden_states = True)
embedds = {}
serialized_dict = {}
for dat in mp:
    for i in dat:
        input_text =  re.sub("\s{2,}", " ", i)
        input_text = input_text.lower()
        print(input_text)
        # print(type(i))
        marked_text = "[CLS] " + input_text + " [SEP]"
        tokenized_text = tokenizer.tokenize(marked_text)
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        tokens_tensor = torch.tensor([indexed_tokens])
        segments_tensors = torch.tensor([segments_ids])
        with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
        token_embeddings = torch.stack(hidden_states, dim=0)
        token_vecs = hidden_states[-2][0]
        sentence_embedding = torch.mean(token_vecs, dim=0)
        # sentence_embedding = sentence_embedding.tolist()
        embed_list = list(sentence_embedding)
        # print(type(embed_list))
        embedds[input_text]=embed_list
for key, value in embedds.items():
    ar = np.array(value)
    serialized_dict[key] = ar.tolist()
# embedds = [len(t) for t in embedds]
save_json(serialized_dict, r'../IEEE_papers/Result/embed.json')

photosensitivity
photosensitive epilepsy
accessibility
federated learning
data heterogeneity
cluster analysis
declarative specification
self service data transformation
programming by example
aesthetic pleasure
aesthetics
validated scale
explainability
neural network architecture search
deep learning
theory
theoretical and empirical research
qualitative study
confidence intervals
bar charts
uncertainty
hierarchical tabular data
tabular visualization
tabular data
music mood classification
ensemble learning
time series visualization
kirigami
physicalization
aesthetics
traces
parallel computing
event sequence visualization
equity
deficit thinking
storytelling
augmented merge tree
scalar field visualization
pixel based visualization
gaussian mixture models
ray casting
scientific visualization
beliefs
cognition
motivated perception
mathematics
physical environmental sciences
tracking transformation
dendrograms
cyber physical networks
human centered computing
molecular dynamics
progressive a

Calculating tfdif scores for all author defined keywords.

In [162]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string 
import itertools

dat = json.load(open(r'../IEEE_papers/Events/articles_w_keywords.json'))
embedds = json.load(open(r'../IEEE_papers/Result/embed.json'))
abstracts = []
keywords = []
scores = {}
for index,datum in enumerate(dat):
    abstract = datum['Abstract'].strip()
    # print(abstract)
    keyword= datum['AuthorKeywords'].strip()
    keyword = keyword.replace("-"," ")
    abstracts.append(abstract)
    keywords.append(keyword)
# save_json(abstracts, r'../IEEE_papers/Result/abs.json')
save_json(keywords, r'../IEEE_papers/Result/keys.json')


tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(abstracts)
scores = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

keyword_scores = {}
for datum in dat:
    abstract = datum['Abstract'].strip()
    keywords= datum['AuthorKeywords'].split(',')
    keys_score = {} 
    for keys in keywords:
        keys = replace_punctuation_with_whitespace(keys)
        key = keys.split()
        score=0.0
        cnt=0
        for k in key:
            k = k.lower()
            if k in scores.keys():
                score += scores[k]
                cnt+=1
            else:
                score = 10
                cnt+=1
        if(cnt!=0) :
            score = score/cnt
        keys_score[keys] = score
    keyword_scores[abstract] = keys_score
test = dict(itertools.islice(keyword_scores.items(), 10))
save_json(keyword_scores, r'../IEEE_papers/Result/final_dict.json')

c:\Users\aryam\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Weighted average using tdidf score, only required when using BERT embeddings.

In [22]:
dat = dat[0:20]
temp=""
for datum in dat:
    abstract = datum['Abstract'].strip()
    keywords= datum['AuthorKeywords'].split(',')
    for keys in keywords:
        count = 0
        for k in key:
            count+=1
            temp = " ".join(k)
            k = k.lower()
            if k in scores.keys():
                score += scores[k]
                cnt+=1
            else:
                score = 10
                cnt+=1
        if(temp):
            if(cnt!=0) :
                score = score/cnt
                embedds[temp]=embedds[temp]*score
                embedds[temp]=np.mean(embedds[temp], axis=0)


Cosine score dictionary and getting top 'n' main participants using tfidf score.

Generate similar words mapping using BERT Embeddings

In [28]:
embedds = json.load(open(r'../IEEE_papers/Result/embed.json'))
cosine_scores = json.load(open(r'../IEEE_papers/Result/cosine.json'))
cosine_dict = {}
check = []
for i,datum in enumerate(flat_list):
        similar = []
        if (flat_list[i] not in check):
                similar.append(flat_list[i])
                check.append(flat_list[i])
        # print(flat_list[i])
        for j in range(i+1,len(flat_list)):
                # print(type(flat_list[i]))
                # print(flat_list[j])
                tup = ','.join([flat_list[i],flat_list[j]])
                # print(type(tup))
                score = cosine_scores[tup]
                if (score>0.83 and flat_list[j] not in check or flat_list[i] not in check):
                                # print("case 1")
                                similar.append(flat_list[j])
                                check.append(flat_list[j])
                                # print("case 2")
                                # similar.append(flat_list[i])
                                # check.append(flat_list[i])
                                # # print("case 3")
                                # similar.append(flat_list[j])
                                # check.append(flat_list[j])
        similar = list(set(similar))
        check = list(set(check))
        print(similar)
        print("$$$")
        post_removed = remove_keywords(similar)
        post_removed.replace(";","")
        new_list = list(post_removed.split(","))

        if(len(new_list)>1):
                mapping = map_listEntities(new_list)
                cosine_dict[post_removed] = mapping.lower()
save_json(cosine_dict, r'../IEEE_papers/Result/map_dict.json')

KeyError: 'photosensitivity,openalex'

Function to get final event graph

In [109]:
data = json.load(r'../IEEE_papers/processed_data/processed_data.json')
from string import punctuation
def post_process_events(dataset):
    for index, datum in enumerate(dataset):
        events = []
        main_characters = []
        datum['doc_id'] = index
        # datum['events_raw'] = datum['events']
        events_str = datum['events'].split('\n')
        for chars in main_participants[index+1]:
            print(chars)
            main_characters.append(chars)
        for event_str in events_str:
            arguments=[]
            event_str = event_str.strip()
            components = event_str.split(':')
            event_type = components[0].strip()
            events.append({'Trigger':event_type, 'Main Participants': main_characters})
        datum['events'] = events
    return dataset

dataset = json.load(open(r'../IEEE_papers/Events/events_merged.json'))
processed_dataset = post_process_events(data)
save_json(processed_dataset, r'../IEEE_papers/Result/final_participants.json')


AttributeError: 'str' object has no attribute 'read'

In [85]:
main_participants

NameError: name 'main_participants' is not defined

In [21]:
f =open('../IEEE_papers/processed_data/processed_data.json')
data = json.load(f)
print(len(data))

3620


In [95]:
# mp = json.load(open(r'../IEEE_papers/Result/main_participants.json'))
# count=0
# for dat in data:
#     keywords = dat['AuthorKeywords']
#     if(keywords == ""):
#         # print(dat)
#         count+=1
#         data.remove(dat)

# for dat in mp:
#     if(dat==""):
#         mp.remove(dat)
# print(count)

605


In [166]:
participants = json.load(open(r'../IEEE_papers/Result/final_dict.json'))
no_keywords = json.load(open(r'../IEEE_papers/Events/articles_w_keywords.json'))
# print(type(participants))
for key, sub_dict in participants.items():
    # print(key)
    top_n_values = get_top_n_values(sub_dict, 3) # change number to change required main participants 
    participants[key]=list(top_n_values.keys())
    # top_n_values_per_key[key] = top_n_values

for i in no_keywords:
    abs = i['Abstract']
    main = participants[abs]
    main = [m.lower() for m in main]
    main = [re.sub(r'\s+', ' ', m) for m in main]
    i['main_participants'] = main

save_json(no_keywords, r'../IEEE_papers/Result/data.json')
        # data['main_participants']=participants[j]

In [158]:
similarity = json.load(open(r'../IEEE_papers/Result/similarity_dict_final.json'))
keys = list(similarity.keys())
print(keys)
similar_mapping = {}
for k in keys:
    components = k.split(",")
    for i in components:
        similar_mapping[i] = similarity[k]
save_json(similar_mapping, r'../IEEE_papers/Result/sim.json')




['data variability,data heterogeneity', 'visual cluster analysis,cluster detection analysis,cluster analysis,cluster comparison', 'declarative specification,declarative grammar', 'aesthetics in visualization,aesthetics', 'explainability,explainable dl', 'neural architecture search,neural network architecture search', 'deep learning based approach,deep learning', 'qualitative evaluation,qualitative experiment,qualitative aspects,quantitative study,qualitative study,qualitative analysis', 'bar charts,bar chart,stacked bar charts', 'uncertainty,uncertainty quantification,implicit uncertainty', 'hierarchical tabular data,tabular data', 'time series data,time varying data visualization,time series analysis,timeseries data analysis,time series visualization,temporal data visualization,time series,multivariate time series,time series clustering,time series exploration,time series segmentation', 'physicalization,physical visualization', 'parallel processing,parallel computing,parallel computat

In [159]:
def check_similarity(keyword):   
    for k in keys:
        if(keyword in k):
            return k
    # return ""

In [160]:
data1 = data[0:20]

In [161]:
main = []
for dat in data1:
    main = dat['main_participants']
    # print(main)
    # components = keys.split(",")
    # print(components)
    for i in main:
        print(i)
        print("$$$$")
        temp = i.lower()
        temp = re.sub(r'\s+', ' ', temp)
        # print(temp)
        # print(test)
        # print(temp)
        if(temp in similar_mapping.keys()):
            mapping = similar_mapping[temp]
            # print(mapping)
            print(temp)
            print("@@@@@")
            print([mapping,i])
            if (mapping not in main):
                dat['main_participants'].append(mapping)
                dat['main_participants'].remove(i)
        # print(dat)
        # print(test)
save_json(data1, r'../IEEE_papers/Result/data1.json')
        

photosensitivity
$$$$
photosensitive epilepsy
$$$$
accessibility
$$$$
federated learning
$$$$
data heterogeneity
$$$$
data heterogeneity
@@@@@
['data variability', 'data heterogeneity']
data variability
$$$$
data variability
@@@@@
['data variability', 'data variability']
declarative specification
$$$$
declarative specification
@@@@@
['declarative specification', 'declarative specification']
self service data transformation
$$$$
programming by example
$$$$
aesthetic pleasure
$$$$
aesthetics
$$$$
aesthetics
@@@@@
['aesthetics in visualization', 'aesthetics']
aesthetics in visualization
$$$$
aesthetics in visualization
@@@@@
['aesthetics in visualization', 'aesthetics in visualization']
explainability
$$$$
explainability
@@@@@
['explainable deep learning', 'explainability']
deep learning
$$$$
deep learning
@@@@@
['deep learning', 'deep learning']
explainable deep learning
$$$$
theory
$$$$
theoretical and empirical research
$$$$
qualitative study
$$$$
qualitative study
@@@@@
['qualitative 

In [137]:
participant_dict = {}
for dat in main:
    for j in dat:
        temp = j.lower()
        temp = re.sub(r'\s+', ' ', temp)
        test = check_similarity(temp)
        if(test):
            mapping = similarity[test]
            print(mapping)

data variability
cluster analysis
declarative specification
aesthetics in visualization
explainable deep learning
neural architecture search
deep learning
cognition theory
qualitative research
bar charts
uncertainty quantification
hierarchical tabular data
hierarchical tabular data
time series analysis
physical visualization
aesthetics in visualization
interaction modeling
parallel processing
event sequence visualization
visual storytelling
scalar field visualization
pixel-based visualization
gaussian mixture models
ray casting
scientific visualization
cognition theory
math visualization
physical and environmental sciences
dendrograms
wireless networks
human computer interaction
molecular visualization
progressive analytics
visual analytics
sports analytics
parallel coordinates
data storytelling
multidimensional data visualization
